My fist project

In [4]:
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import LabelEncoder


def data_preparing(data):
    y = []
    x = data
    
    if 'Survived' in data:
        y = data['Survived']
        x = data.drop(columns=['Survived'])
        
    lbEncoder = LabelEncoder()

    lbEncoder.fit(x.Sex)
    x['Sex'] = lbEncoder.transform(x.Sex)


    lbEncoder.fit(x.Name)
    x['Name'] = lbEncoder.transform(x.Name)

    lbEncoder.fit(x.Ticket)
    x['Ticket'] = lbEncoder.transform(x.Ticket)
    
    x['Age'] = x['Age'].fillna(x['Age'].mean())
    
    x = x.drop('Cabin',1)
    
    #Embarked
    maxEmbarked = x.groupby('Embarked').count()['PassengerId']
    tmp = maxEmbarked[maxEmbarked == maxEmbarked.max()].index[0]
    x.fillna(tmp)


    x['Embarked'] = x['Embarked'].astype('str')


    lbEncoder.fit(x.Embarked)
    x['Embarked'] = lbEncoder.transform(x.Embarked)
    
    #??
    tmp = x.Fare.median()
    x['Fare'].fillna(tmp)
    
    return x,y

In [5]:

data_train = pd.read_csv("./kaggle/input/titanic/train.csv")

x_train, y_train = data_preparing(data_train)

x_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,3,108,1,22.0,1,0,523,7.2500,2
1,2,1,190,0,38.0,1,0,596,71.2833,0
2,3,3,353,0,26.0,0,0,669,7.9250,2
3,4,1,272,0,35.0,1,0,49,53.1000,2
4,5,3,15,1,35.0,0,0,472,8.0500,2


In [6]:
data_test = pd.read_csv("./kaggle/input/titanic/test.csv")
x_test, y_test = data_preparing(data_test)
x_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,206,1,34.5,0,0,152,7.8292,1
1,893,3,403,0,47.0,1,0,221,7.0000,2
2,894,2,269,1,62.0,0,0,73,9.6875,1
3,895,3,408,1,27.0,0,0,147,8.6625,2
4,896,3,178,0,22.0,1,1,138,12.2875,2
